In [ ]:
import os
import re
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize
from tqdm.auto import tqdm
nltk.download('punkt')

!pip install --upgrade spicy
!python -m spacy download fr_core_news_md
import fr_core_news_md
nlp = fr_core_news_md.load()

In [ ]:
#process a sentence using the model
doc = nlp("je tu aime")
# It's that simple - all of the vectors and words are assigned after this point
# Get the vector for 'aime':
doc[2].vector
# Get the mean vector for the entire sentence (useful for sentence classification etc.)
doc.vector

In [16]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/ВКР/far new proga/corpora.csv')
data

,sentence,label,dependency structure,tree structure
0,Que c'était super ?,dyadic clarification,"[[{'id': 1, 'text': 'Que', 'lemma': 'que', 'up...","[""(SENT (Ssub (CS Que) (Sint (VN (CLS c') (V é..."
1,Si je J'étais responsable.,dyadic,"[[{'id': 1, 'text': 'Si', 'lemma': 'si', 'upos...","[""(SENT (Ssub (CS Si) (Sint (VN (CLS je) (CLS ..."
2,Parce que tu ne sais pas tirer.,dyadic,"[[{'id': 1, 'text': 'Parce', 'lemma': 'parce',...",['(SENT (Ssub (CS+ (CS Parce) (CS que)) (Sint ...
3,Comme si je ne redoutais pas assez de devoir l...,dyadic,"[[{'id': 1, 'text': 'Comme', 'lemma': 'comme',...",['(SENT (Ssub (CS+ (ADV Comme) (CS si)) (Sint ...
4,Si c'est un garcon.,dyadic,"[[{'id': 1, 'text': 'Si', 'lemma': 'si', 'upos...","[""(SENT (Ssub (CS Si) (Sint (VN (CLS c') (V es..."
...,...,...,...,...
1481,Que je vous regarde un peu.,wish,"[[{'id': 1, 'text': 'Que', 'lemma': 'que', 'up...",['(SENT (Ssub (CS Que) (Sint (VN (CLS je) (CLO...
1482,Que tu gagnes ou que tu perdes.,wish,"[[{'id': 1, 'text': 'Que', 'lemma': 'que', 'up...",['(SENT (Ssub (CS Que) (Sint (VN (CLS tu) (V g...
1483,Pour qu'ils tuent en chantant des hymnes plein...,wish,"[[{'id': 1, 'text': 'Pour', 'lemma': 'pour', '...","[""(SENT (Ssub (CS+ (P Pour) (CS qu')) (Sint (V..."
1484,Qu'elle ne le fasse pas.,wish,"[[{'id': 1, 'text': ""Qu'"", 'lemma': 'que', 'up...","[""(SENT (Ssub (CS Qu') (VN (CLS elle) (ADV ne)..."


In [17]:
data['label'] = data['label'].replace(['dyadic '],'dyadic')
data['label'] = data['label'].replace(['wish '],'wish')

In [18]:
data.label.unique()

array(['dyadic clarification', 'dyadic', 'dyadic complaint', 'ERROR',
       'memory', 'offer', 'polite discourse formulas', 'polite request',
       'regret/unrealizable desire', 'threat', 'what happens', 'wish'],
      dtype=object)

In [19]:
import sqlite3 as sql
conn = sql.connect('insub_corpora.db')
data.to_sql('insub_corpora', conn)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [ ]:
si = []
que = []
comme = []
lorsque = []
quand = []
sinon = []
puisque = []

for i in range(len(data)):
  for sent in eval(data['dependency structure'][i]):
    for word in sent:
      try:
        if word['upos'] == 'SCONJ':
          if word['lemma'] == 'si':
            si.append(i)
            break
          if word['lemma'] == 'que':
            que.append(i)
            break
          if word['lemma'] == 'comme':
            comme.append(i)
            break
          if word['lemma'] == 'lorsque':
            lorsque.append(i)
            break
          if word['lemma'] == 'quand':
            quand.append(i)
            break
          if word['lemma'] == 'sinon':
            sinon.append(i)
            break
          if word['lemma'] == 'puisque':
            puisque.append(i)
            break
      except:
        continue

print('si-----',len(si))
print('que-----',len(que))
print('comme-----',len(comme))
print('lorsque-----',len(lorsque))
print('quand-----',len(quand))
print('sinon-----',len(sinon))
print('puisque-----',len(puisque))

si----- 373
que----- 900
comme----- 122
lorsque----- 1
quand----- 82
sinon----- 0
puisque----- 9


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.unique())

LabelEncoder()

In [ ]:
data['target'] = le.transform(data['label'])
data.head(2)

,sentence,label,dependency structure,tree structure,target
0,Que c'était super ?,dyadic clarification,"[[{'id': 1, 'text': 'Que', 'lemma': 'que', 'up...","[""(SENT (Ssub (CS Que) (Sint (VN (CLS c') (V é...",2
1,Si je J'étais responsable.,dyadic,"[[{'id': 1, 'text': 'Si', 'lemma': 'si', 'upos...","[""(SENT (Ssub (CS Si) (Sint (VN (CLS je) (CLS ...",1


In [ ]:
import spacy
import string
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

In [ ]:
# Load English tokenizer, tagger, parser, NER and word vectors
parser = French()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [ ]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [ ]:
data_new = data.loc[data['label'].isin(['memory', 'offer', 'polite discourse formulas', 'polite request',
       'regret/unrealizable desire', 'threat', 'what happens', 'wish'])]
data_new.head()

,sentence,label,dependency structure,tree structure,target
1183,Quand je pense que ses doigts m'ont touchée !,memory,"[[{'id': 1, 'text': 'Quand', 'lemma': 'quand',...","[""(SENT (Ssub (CS Quand) (Sint (VN (CLS je) (V...",4
1184,"Quand, à la fin, j'ai trouvé celui de l'immort...",memory,"[[{'id': 1, 'text': 'Quand', 'lemma': 'quand',...","[""(SENT (Ssub (CS Quand) (Sint (PONCT ,) (ADV+...",4
1185,Quand elle recrutait ces étudiants pour être d...,memory,"[[{'id': 1, 'text': 'Quand', 'lemma': 'quand',...",['(SENT (Ssub (CS Quand) (Sint (VN (CLS elle) ...,4
1186,Quand tu étais sûr qu'il vaincrait Bartholomew.,memory,"[[{'id': 1, 'text': 'Quand', 'lemma': 'quand',...","[""(SENT (Ssub (CS Quand) (Sint (VN (CLS tu) (V...",4
1187,Quand on parle d'authenticité !,memory,"[[{'id': 1, 'text': 'Quand', 'lemma': 'quand',...","[""(SENT (Ssub (CS Quand) (Sint (VN (CLS on) (V...",4


In [ ]:
from sklearn.model_selection import train_test_split

X = data['sentence'] # the features we want to analyze
ylabels = data['target'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, stratify= ylabels, shuffle=True)

In [ ]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
#classifier = LogisticRegression()

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3)

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7fa14aa87310>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x7fa14b9e3b90>,
                                 vocabulary=None)),
                ('classifier',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,


In [ ]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

In [ ]:
# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted, average='micro'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted, average='micro'))

Logistic Regression Accuracy: 0.6677852348993288
Logistic Regression Precision: 0.6677852348993288
Logistic Regression Recall: 0.6677852348993288


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, predicted, average='micro')

0.6677852348993288

In [ ]:
from sklearn.metrics import classification_report
target_names = data.label.unique()
print(classification_report(y_test, predicted, target_names=target_names))

                            precision    recall  f1-score   support

      dyadic clarification       1.00      0.15      0.27        13
                    dyadic       0.68      0.96      0.80       190
          dyadic complaint       0.50      0.04      0.08        24
                     ERROR       0.00      0.00      0.00        10
                    memory       0.00      0.00      0.00         2
                     offer       0.00      0.00      0.00         4
 polite discourse formulas       0.50      0.64      0.56        11
            polite request       0.00      0.00      0.00         1
regret/unrealizable desire       0.50      0.14      0.22         7
                    threat       0.00      0.00      0.00         1
              what happens       0.00      0.00      0.00        14
                      wish       0.55      0.29      0.37        21

                  accuracy                           0.67       298
                 macro avg       0.31      0.1

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
